In [3]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  1


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, BatchNormalization
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [5]:
df=pd.read_csv("link to the filetered data")

In [6]:
texts = df['after_filter'].values
sentiments = df['sentiment'].values



In [7]:
label_encoder = LabelEncoder()
sentiments = label_encoder.fit_transform(sentiments)

In [8]:
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)


In [9]:
sequences[0]

[1576,
 749,
 31,
 136,
 7025,
 206,
 2500,
 55,
 388,
 363,
 32,
 2538,
 4205,
 28,
 357,
 101,
 55,
 419,
 1334,
 6238,
 1803,
 8676,
 2009,
 4589,
 4420,
 1101,
 193,
 357,
 3171,
 173,
 159,
 419,
 233,
 8804,
 158,
 2086,
 1054,
 3752,
 1922,
 935,
 5573,
 2711,
 9046,
 203,
 143,
 3964,
 167,
 2275,
 169,
 6965,
 5983,
 9434,
 6703,
 72,
 84,
 114,
 973,
 52,
 102,
 106,
 2422,
 555,
 49,
 1176,
 3340,
 1957,
 887,
 555,
 1091,
 555,
 614,
 7025,
 655,
 206,
 60,
 2538,
 1247,
 1733,
 1204,
 118,
 1057,
 957,
 50,
 7745,
 143,
 1573,
 1682,
 357,
 357,
 5984,
 5937,
 3998,
 2322,
 5379,
 297,
 414,
 84,
 5890,
 444,
 463,
 5379,
 412,
 935,
 351,
 927,
 1542,
 935,
 336,
 31,
 3003,
 2596,
 531,
 921,
 3299]

In [10]:

# Padding
max_len = 100  # Define max length for padding
padded_sequences = pad_sequences(sequences, maxlen=max_len)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, sentiments, test_size=0.2, random_state=42)


In [11]:
y_train = np.array(y_train)
y_test = np.array(y_test)


In [12]:
y_train

array([0, 0, 1, ..., 0, 1, 1])

In [13]:
input_dim = 10000  # Vocabulary size
output_dim = 128  # Embedding dimensions
max_len = 100

In [14]:
from tensorflow.keras.optimizers import Adam

from keras.callbacks import EarlyStopping
from keras.regularizers import l2

In [15]:
from tensorflow.keras.layers import Embedding, Flatten

In [20]:
model = Sequential()
model.add(Embedding(input_dim=input_dim, output_dim=output_dim, input_length=max_len))
model.add(LSTM(64))  # LSTM layer
model.add(Dropout(0.3))  # Dropout for regularization
model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.01)))  # L2 Regularization
model.add(BatchNormalization())  # Batch Normalization
model.add(Dropout(0.3))  # Reduced Dropout
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01)))  # L2 Regularization
model.add(BatchNormalization())  # Batch Normalization
model.add(Dropout(0.3))  # Reduced Dropout
model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification

model.compile(optimizer=Adam(learning_rate=0.01), loss='binary_crossentropy', metrics=['accuracy'])

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_accuracy', patience=3)

# Model summary
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 100, 128)          1280000   
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_3 (Dense)             (None, 128)               8320      
                                                                 
 batch_normalization_2 (Bat  (None, 128)               512       
 chNormalization)                                                
                                                                 
 dropout_4 (Dropout)         (None, 128)               0         
                                                      

In [21]:
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2,callbacks=[early_stopping])


Epoch 1/50
1000/1000 [==============================] - 47s 43ms/step - loss: 0.6949 - accuracy: 0.7254 - val_loss: 0.5048 - val_accuracy: 0.8295
Epoch 2/50
1000/1000 [==============================] - 15s 15ms/step - loss: 0.4200 - accuracy: 0.8664 - val_loss: 0.4400 - val_accuracy: 0.8506
Epoch 3/50
1000/1000 [==============================] - 12s 12ms/step - loss: 0.3650 - accuracy: 0.8855 - val_loss: 0.4504 - val_accuracy: 0.8426
Epoch 4/50
1000/1000 [==============================] - 11s 11ms/step - loss: 0.3333 - accuracy: 0.8992 - val_loss: 0.4648 - val_accuracy: 0.8585
Epoch 5/50
1000/1000 [==============================] - 9s 9ms/step - loss: 0.3146 - accuracy: 0.9047 - val_loss: 0.4429 - val_accuracy: 0.8494
Epoch 6/50
1000/1000 [==============================] - 9s 9ms/step - loss: 0.3068 - accuracy: 0.9085 - val_loss: 0.4396 - val_accuracy: 0.8531
Epoch 7/50
1000/1000 [==============================] - 10s 10ms/step - loss: 0.2996 - accuracy: 0.9108 - val_loss: 0.4243 - val

In [22]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy}")


313/313 [==============================] - 3s 7ms/step - loss: 0.4284 - accuracy: 0.8492
Test Accuracy: 0.8492000102996826
